In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so
import plotly.express as px
pd.set_option('display.max_rows', None)
#Import necessary libraries for visualizing and dataframes

In [2]:
#change device01 to desired device
path="/srv/data/my_shared_data_folder//iet_2022/device01/" 

#DO NOT CHANGE THE FOLLOWING "COLUMNS" LIST 
columns = ['time','deviceid','tool','direction','protocol','target','pktloss','method','zip','isp','value','topic','annonipaddr','ipaddrchanged']

In [3]:
#all_files is a list of device data by month, change the device number based on desired device
all_files1 = ["device01_netrics_latency_202207.csv","device01_netrics_latency_202202.csv", "device01_netrics_latency_202208.csv",
            "device01_netrics_latency_202203.csv",  "device01_netrics_latency_202209.csv",
            "device01_netrics_latency_202204.csv",  "device01_netrics_latency_202210.csv",
            "device01_netrics_latency_202205.csv",  "device01_netrics_latency_202211.csv",
            "device01_netrics_latency_202206.csv",  "device01_netrics_latency_202212.csv"]

In [4]:
#Change the device within the csv path accordingly
initialdata = pd.read_csv(path + 'device01_netrics_latency_202201.csv',names=columns)

In [5]:
def dfmaker(listoffiles, initialdata, path, columns):
    '''
    Takes the list 'all_files' and reads
    each csv file and concatenates it into one dataframe.
        
    Then creates several columns: "hour", "date", "month", and "anomalytotal"
    '''
    devicefulldf = initialdata
    for file in listoffiles:       # <----creates a for loop that adds all the csv files from the 'all_files' list and adds it to a new dataframe called devicefulldf
        df = pd.read_csv(path + file, names=columns)
        devicefulldf = pd.concat([devicefulldf,df])
        
    devicefulldf = devicefulldf.sort_values(by=['time'])      # <----Sorting devicefulldf into chronological order based off the "time" column
    devicefulldf = devicefulldf.loc[devicefulldf['method'] == 'avg']      #<---- Selecting the statsistcal method 'avg' from the method coloumn, change accordingly based on desired method
    devicefulldf = devicefulldf.loc[devicefulldf['target'].isin(['Atlanta', 'Seattle', 'Chicago'])]   #<---- locating the three servers from the 'target' column' within devicefulldf, change accordingly based on desired server(s)
    
    return devicefulldf
 #Created functions to split the time column by date, time, & month in order to isolate those values
def split_hour(x):
    row = x
    return row[11:13]

def split_date(x):
    row = x
    return row[5:10]

def split_month(x):
    row = x
    return row[5:7]

#Created a function that manually detecs anomalies, change accordingly based on desired algorithim and or manual threshold 
def anomaly_detec(z):
    stdev_threshold = z.std() + z.mean()
    anomaly_count = z >= stdev_threshold
    return anomaly_count

def groupbyfunc(devicefulldf):
    '''
    This groups the dataframe based on "month" and server"
    '''
    deviceanomaly_df = devicefulldf.groupby(['month','date','target'])[['anomalytotal']].sum()
    anomaly_by_month_df= deviceanomaly_df.groupby(['month','target'])[['anomalytotal']].median().reset_index()
    return anomaly_by_month_df

In [6]:
def main(listoffiles):
    devicefulldf = dfmaker(listoffiles, initialdata, path, columns)
    
    devicefulldf['hour'] = devicefulldf['time'].apply(split_hour)
    devicefulldf['date'] = devicefulldf['time'].apply(split_date)
    devicefulldf['month'] = devicefulldf['time'].apply(split_month)
    devicefulldf["anomalytotal"] = anomaly_detec(devicefulldf['value'])
    
    #return devicefulldf["month"].unique()
    
    anomaly_by_month_df = groupbyfunc(devicefulldf)
    
    return anomaly_by_month_df

In [67]:
###everything is good above here YIPPEEE###

In [7]:
main(all_files1)

,month,target,anomalytotal
0,01,Atlanta,0.0
1,01,Chicago,0.0
2,01,Seattle,288.0
3,02,Atlanta,0.0
4,02,Chicago,0.0
5,02,Seattle,288.0
6,03,Atlanta,0.0
7,03,Chicago,0.0
8,03,Seattle,288.0
9,04,Atlanta,0.0
